# 문자 단위 RNN 언어 모델

### 문자 단위 RNN 언어 모델 구현하기 (다 대 다 LSTM)

**1) 데이터에 대한 이해와 전처리**

In [45]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

'이상한 나라의 앨리스' 소설 데이터를 로드하고 특수문자를 제거하고 단어를 소문자화하는 간단한 전처리를 수행

In [46]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
sentences = []
for sentence in f: # 데이터를 한 줄씩 읽는다.
    sentence = sentence.strip() # strip()을 통해 \r, \n을 제거한다.
    sentence = sentence.lower() # 소문자화.
    sentence = sentence.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

전처리된 결과인 리스트에서 5개의 원소만 출력하면 이들은 문자열로 구성되어 있으므로 하나의 문자열로 통합하도록 함

In [47]:
sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

하나의 문자열로 통합된 길이는 15만 9천자

In [48]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 글자의 개수: 159484


In [49]:
print(total_data[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


문자열로부터 단어 집합이 아닌, 문자 집합을 만들도록 함

In [50]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 56


정수, 특수문자, 구두점, 알파벳을 포함하는 문자 집합의 각 문자에 정수를 부여

In [51]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


정수로부터 문자를 리턴하는 함수 생성

In [52]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

15만 9천의 길이의 문자열로부터 샘플을 만들기 위해 문장 길이를 60으로 정하고 60으로 나누어 2658개의 샘플을 만듦

In [53]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(total_data) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2658


전처리를 위해 문장을 정수 인코딩 후 오른쪽으로 한 칸 쉬프트된 문장으로 바꾸어 주도록 함

In [54]:
train_X = []
train_y = []

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 픽한다.
    X_sample = total_data[i * seq_length: (i + 1) * seq_length]

    # 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [55]:
print('X 데이터의 첫번째 샘플 :',train_X[0])
print('y 데이터의 첫번째 샘플 :',train_y[0])
print('-'*50)
print('X 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_X[0]])
print('y 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_y[0]])

X 데이터의 첫번째 샘플 : [49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
y 데이터의 첫번째 샘플 : [37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
--------------------------------------------------
X 데이터의 첫번째 샘플 디코딩 : ['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
y 데이터의 첫번째 샘플 디코딩 : ['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e',

In [56]:
print(train_X[1])

[43, 33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54]


In [57]:
print(train_y[1])

[33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54, 52]


문자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않으므로 전체 데이터 뿐만 아니라 입력 시퀀스에 대해서도 원-핫 인코딩을 수행

In [58]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [59]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


**2) 모델 설계하기**

하이퍼파라미터인 은닉 상태의 크기는 256로 하고 다 대 다 구조의 LSTM을 사용하며 LSTM 은닉층은 두 개를 사용함<br>
그리고 전결합층을 출력층으로 하여 문자 집합 크기만큼의 뉴런을 배치하여 모델을 설계하도록 함<br>
해당 모델은 모든 시점에서 모든 가능한 문자 중 하나의 문자를 예측하는 다중 클래스 분류 문제를 수행하는 것이므로<br>
출력층에서 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로는 크로스 엔트로피 함수를 사용하여 80 에포크 수행

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, None, 256)         320512    
                                                                 
 lstm_4 (LSTM)               (None, None, 256)         525312    
                                                                 
 time_distributed_1 (TimeDis  (None, None, 56)         14392     
 tributed)                                                       
                                                                 
Total params: 860,216
Trainable params: 860,216
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 4s - loss: 3.0669 - accuracy: 0.1830 - 4s/epoch - 42ms/step
Epoch 2/80
84/84 - 1s - loss: 2.7372 - accuracy: 0.2457 - 799ms/epoch - 10ms/step
Epoch 3/80
84/84 - 1s - loss: 2.3997 - accuracy: 0.3264 - 838ms/epoch - 10ms/step
Epoch 4/80
84/84 - 1s - loss: 2.2594 - accuracy: 0.3560 - 811ms/epoch - 10ms/step
Epoch 5/80
84/84 - 1s - loss: 2.1568 - accuracy: 0.3840 - 834ms/epoch - 10ms/step
Epoch 6/80
84/84 - 1s - loss: 2.0705 - accuracy: 0.4050 - 833ms/epoch - 10ms/step
Epoch 7/80
84/84 - 1s - loss: 2.0000 - accuracy: 0.4236 - 794ms/epoch - 9ms/step
Epoch 8/80
84/84 - 1s - loss: 1.9450 - accuracy: 0.4382 - 807ms/epoch - 10ms/step
Epoch 9/80
84/84 - 1s - loss: 1.8941 - accuracy: 0.4515 - 815ms/epoch - 10ms/step
Epoch 10/80
84/84 - 1s - loss: 1.8482 - accuracy: 0.4638 - 793ms/epoch - 9ms/step
Epoch 11/80
84/84 - 1s - loss: 1.8076 - accuracy: 0.4741 - 819ms/epoch - 10ms/step
Epoch 12/80
84/84 - 1s - loss: 1.7707 - accuracy: 0.4853 - 824ms/epoch - 10ms/step
Epoch 13/80
84/84 

특정 문자를 주면 다음 문자를 계속해서 생성해내는 함수를 구현하고 인자로 학습 모델, 문자를 몇 번 생성할 것인지 횟수를 전달하면<br> 
해당 함수는 임의로 시작 문자를 정한 뒤 정해진 횟수만큼 다음 문자를 지속적으로 예측하여 문장을 생성함

In [62]:
def sentence_generation(model, length):
    # 글자에 대한 랜덤 인덱스 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤 익덱스로부터 글자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [63]:
result = sentence_generation(model, 100)
print(result)

38 번 글자 i 로 예측을 시작!
1/1 [==============================] - 0s 21ms/step
ing voice. theres more evidence to come yet, please your maje_ty, said the king, looking round one, a


# 문자 단위 RNN으로 텍스트 생성하기

### 문자 단위 RNN 언어 모델 구현하기 (다 대 일 LSTM)

**1) 데이터에 대한 이해와 전처리**

In [64]:
import numpy as np
from tensorflow.keras.utils import to_categorical

엉터리 노래 가사를 로드

In [65]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장 후 문자 집합을 생성

In [66]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [67]:
# 중복을 제거한 글자 집합 생성
char_vocab = sorted(list(set(raw_text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [68]:
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


대, 소문자 알파벳 또는 구두점 등의 단위의 집합인 문자 집합이 만들어지면 각 문자에 정수를 부여하도록 함

In [69]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


입력 시퀀스의 길이가 10이 되도록 데이터를 구성하며 예측 대상인 문자도 필요하므로 길이가 11이 되도록 구성

In [70]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


총 샘플의 수는 426개가 되므로 10개를 출력하면 엉터리 노래의 첫 문장이 10개의 샘플로 분리된 것을 확인할 수 있음

In [71]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

전처리를 위해 문자 집합에 정수 인덱스를 부여한 것을 가지고 전체 데이터에 대해 정수 인코딩을 수행

In [72]:
encoded_sequences = []
for sequence in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    encoded_sequence = [char_to_index[char] for char in sequence] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    encoded_sequences.append(encoded_sequence)

In [73]:
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

예측 대상인 문자를 분리시켜주기 위해 모든 샘플 문장에 대해서 마지막 문자를 분리

In [74]:
encoded_sequences = np.array(encoded_sequences)
X_data = encoded_sequences[:,:-1]

# 맨 마지막 위치의 글자를 분리
y_data = encoded_sequences[:,-1]

In [75]:
print(X_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


정상적으로 분리가 되었다면 분리된 샘플과 마지막 문자에 대해서 원-핫 인코딩을 수행

In [76]:
# 원-핫 인코딩
X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

In [77]:
print(X_data_one_hot.shape)

(426, 10, 33)


**2) 모델 설계하기**

In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

하이퍼파라미터인 은닉 상태의 크기는 64로 하고 다 대 일 구조의 LSTM을 사용<br>
그리고 전결합층을 출력층으로 하여 문자 집합 크기만큼의 뉴런을 배치하여 모델을 설계하도록 함<br>
해당 모델은 마지막 시점에서 모든 가능한 문자 중 하나의 문자를 예측하는 다중 클래스 분류 문제를 수행하는 것이므로<br>
출력층에서 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로는 크로스 엔트로피 함수를 사용하여 100 에포크 수행

In [79]:
hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [80]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

Epoch 1/100
14/14 - 1s - loss: 3.4742 - accuracy: 0.1103 - 1s/epoch - 105ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3409 - accuracy: 0.1948 - 66ms/epoch - 5ms/step
Epoch 3/100
14/14 - 0s - loss: 3.0741 - accuracy: 0.1972 - 49ms/epoch - 4ms/step
Epoch 4/100
14/14 - 0s - loss: 3.0021 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9626 - accuracy: 0.1972 - 50ms/epoch - 4ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9545 - accuracy: 0.1972 - 56ms/epoch - 4ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9436 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 8/100
14/14 - 0s - loss: 2.9256 - accuracy: 0.1972 - 49ms/epoch - 4ms/step
Epoch 9/100
14/14 - 0s - loss: 2.9147 - accuracy: 0.1972 - 50ms/epoch - 4ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8979 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8862 - accuracy: 0.1972 - 49ms/epoch - 4ms/step
Epoch 12/100
14/14 - 0s - loss: 2.8614 - accuracy: 0.1972 - 58ms/epoch - 4ms/step
Epoch 13/100
14/14 - 0s -

그 후 문자열을 주면 다음 문자를 계속해서 예측하는 것을 반복하여 최종적으로 문장을 완성시키는 함수를 생성

In [81]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):

    # 초기 시퀀스
    init_text = seed_text
    sentence = ''

    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        for char, index in char_to_index.items():
            if index == result:
                break

        # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        seed_text = seed_text + char

        # 예측 글자를 문장에 저장
        sentence = sentence + char

    sentence = init_text + sentence
    return sentence

출력된 두 문장을 연속적으로 훈련 데이터에서 나온 적이 없는 문장임에도 모델이 임의로 생성해낸 것을 볼 수 있음

In [82]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to han wort Ialt to bee.. But whee I stop lo ta
